In [26]:
import gensim.downloader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
from gensim.models import KeyedVectors
from keras.layers import Dense, Flatten, LSTM, GRU, Bidirectional, Embedding
from keras.models import Sequential
from keras.preprocessing import sequence, text
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, classification_report
from sklearn.model_selection import train_test_split
from spacy.tokenizer import Tokenizer

In [27]:
data = pd.read_csv('data\preprocessed.csv').dropna()
data = data[((data[" 'American'"]==1) | (data["'American'"]==1))]

In [28]:
nlp = spacy.load("en_core_web_sm")
#glove = gensim.downloader.load('glove-twitter-200')
#glove.save("glove.model")
glove = KeyedVectors.load("glove.model")

In [29]:
def get_vector(sent):
    for word in sent.lower().split(' '):
        if word.isalpha() == True:
            try:
                return glove[word].tolist()
            except KeyError:
                return glove['unk'].tolist()
        else:
            return glove['unk'].tolist()

In [30]:
def pad_trunc(data, maxlen):
    new_data = []
    zero_vector = []
    for _ in range(len(data[10][0])):
        zero_vector.append(0.0)
    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
        
    return new_data

In [31]:
data['vectors'] =  data['Reviews_cleaned'].apply(lambda x: [get_vector(x)])

In [32]:
X = data['vectors']
y = data["Category"]
X = np.array(pad_trunc(X, 3))

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [34]:
X_train.shape, X_test.shape, X.shape

((1460, 3, 200), (366, 3, 200), (1826, 3, 200))

In [35]:
model = Sequential()
model.add(Bidirectional(LSTM(32, return_sequences=True, input_shape=(3,200), dropout=0.2, recurrent_dropout=0.2)))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=16,
                    validation_split=0.2)

Epoch 1/10
73/73 [==============================] - 12s 35ms/step - loss: 0.6436 - accuracy: 0.6952 - val_loss: 0.5996 - val_accuracy: 0.7329
Epoch 2/10
73/73 [==============================] - 1s 12ms/step - loss: 0.5368 - accuracy: 0.7346 - val_loss: 0.5054 - val_accuracy: 0.7329
Epoch 3/10
73/73 [==============================] - 1s 13ms/step - loss: 0.5116 - accuracy: 0.7346 - val_loss: 0.5045 - val_accuracy: 0.7329
Epoch 4/10
73/73 [==============================] - 1s 13ms/step - loss: 0.5119 - accuracy: 0.7346 - val_loss: 0.5078 - val_accuracy: 0.7329
Epoch 5/10
73/73 [==============================] - 1s 12ms/step - loss: 0.5143 - accuracy: 0.7346 - val_loss: 0.5034 - val_accuracy: 0.7329
Epoch 6/10
73/73 [==============================] - 1s 12ms/step - loss: 0.5126 - accuracy: 0.7346 - val_loss: 0.5047 - val_accuracy: 0.7329
Epoch 7/10
73/73 [==============================] - 1s 12ms/step - loss: 0.5106 - accuracy: 0.7346 - val_loss: 0.5039 - val_accuracy: 0.7329
Epoch 8/10
7

In [37]:
y_pred = np.round(model.predict(X_test))
cohen_kappa_score(y_test, y_pred)

0.46407875880117144

In [38]:
f1_score(y_test, y_pred)

0.8065843621399177

In [39]:
model = Sequential()
model.add(Bidirectional(GRU(32, return_sequences=True, input_shape=(3,200), dropout=0.2, recurrent_dropout=0.2)))
model.add(GRU(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=16,
                    validation_split=0.2)

Epoch 1/10
73/73 [==============================] - 12s 49ms/step - loss: 0.5862 - accuracy: 0.7080 - val_loss: 0.5073 - val_accuracy: 0.7329
Epoch 2/10
73/73 [==============================] - 1s 15ms/step - loss: 0.5140 - accuracy: 0.7346 - val_loss: 0.5102 - val_accuracy: 0.7329
Epoch 3/10
73/73 [==============================] - 1s 14ms/step - loss: 0.5149 - accuracy: 0.7346 - val_loss: 0.5048 - val_accuracy: 0.7329
Epoch 4/10
73/73 [==============================] - 1s 13ms/step - loss: 0.5126 - accuracy: 0.7346 - val_loss: 0.5037 - val_accuracy: 0.7329
Epoch 5/10
73/73 [==============================] - 1s 15ms/step - loss: 0.5120 - accuracy: 0.7346 - val_loss: 0.5034 - val_accuracy: 0.7329
Epoch 6/10
73/73 [==============================] - 1s 14ms/step - loss: 0.5116 - accuracy: 0.7346 - val_loss: 0.5042 - val_accuracy: 0.7329
Epoch 7/10
73/73 [==============================] - 2s 22ms/step - loss: 0.5117 - accuracy: 0.7346 - val_loss: 0.5033 - val_accuracy: 0.7329
Epoch 8/10
7

In [40]:
y_pred = np.round(model.predict(X_test))
cohen_kappa_score(y_test, y_pred)

0.46407875880117144

In [41]:
f1_score(y_test, y_pred)

0.8065843621399177